In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

path='CSV_files/'

orders=pd.read_csv(path+ 'olist_orders_dataset.csv',
                   parse_dates=['order_purchase_timestamp',
                                'order_delivered_customer_date',
                                'order_estimated_delivery_date',
                               'order_delivered_carrier_date',
                               'order_approved_at'])

customers=pd.read_csv(path + 'olist_customers_dataset.csv')

items=pd.read_csv(path+ 'olist_order_items_dataset.csv')

products= pd.read_csv(path+ 'olist_products_dataset.csv')

categories= pd.read_csv(path+ 'product_category_name_translation.csv')

print("orders loaded successfully")
display(orders.head())
# display(products.head())
# display(items.tail())
# display(customers.head())
# display(categories.head())
# display(orders.tail())

orders loaded successfully


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [4]:
print(orders.isnull().sum())
orders.info()

# orders['order_approved_at']=pd.to_datetime(orders['order_approved_at'],errors='coerce')

# orders['approval_delay']=(orders['order_approved_at']-orders['order_purchase_timestamp']).dt.total_seconds()/3600
orders['approval_delay']=(orders['order_approved_at']-orders['order_purchase_timestamp']).dt.total_seconds()/3600

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  964

In [5]:
orders.info()

customers['customer_zip_code_prefix']=customers['customer_zip_code_prefix'].astype(str)
customers.info()

items
items['shipping_limit_date']=pd.to_datetime(items['shipping_limit_date'])

items['order_id']=items['order_id'].astype(str)
items['product_id']=items['product_id'].astype(str)
items['seller_id']=items['seller_id'].astype(str)

items['total_order_value']=items['price']+items['freight_value']
items.info()
items
print(items.isnull().sum())
print(customers.isnull().sum())

items.rename(columns={'total_order_value':'line_item_total'},inplace=True)


order_totals=items.groupby('order_id')['line_item_total'].sum().reset_index()
order_totals.columns=['order_id','total_order_value']

order_totals
items
# products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   approval_delay                 99281 non-null  float64       
dtypes: datetime64[ns](5), float64(1), object(3)
memory usage: 6.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data col

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,line_item_total
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,218.04
...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,343.40
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,386.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,116.85
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,64.71


In [6]:
products

products['product_category_name']=products['product_category_name'].fillna('others')
products['product_photos_qty']=products['product_photos_qty'].fillna(0)


products['product_name_lenght']=products['product_name_lenght'].fillna(0)
products['product_description_lenght']=products['product_description_lenght'].fillna(0)

cols_to_fix=['product_name_lenght','product_description_lenght','product_photos_qty']
products[cols_to_fix]=products[cols_to_fix].astype(int)

products.isnull().sum()
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32951 non-null  object 
 2   product_name_lenght         32951 non-null  int64  
 3   product_description_lenght  32951 non-null  int64  
 4   product_photos_qty          32951 non-null  int64  
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(4), int64(3), object(2)
memory usage: 2.3+ MB


In [7]:
categories
categories.info()
categories.isnull().sum()
categories

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [23]:
# merging and new columns to understand delivery

# customers
# orders
# order_totals

orders_master=orders.merge(order_totals, on='order_id',how='left')\
                     .merge(customers, on='customer_id',how='left')

orders_master['total_order_value']=orders_master['total_order_value'].fillna(0)
if orders_master['customer_city'].isnull().any():
    print("Warning!! some orders are missing customer location data")

orders_master['promised_delivery_time']=(orders_master['order_estimated_delivery_date']-orders_master['order_purchase_timestamp']).dt.days
orders_master['delivery_time']=(orders_master['order_delivered_customer_date']-orders_master['order_purchase_timestamp']).dt.days




In [46]:
# first high level insights
# 1. State revenue
state_revenue=orders_master.groupby('customer_state')['total_order_value'].sum().sort_values(ascending=False).reset_index()
print(state_revenue.head(10))

state_revenue.columns=['State','Total_Revenue']
state_revenue

# 2. Order Status Distribution
status_summary=(orders_master['order_status'].value_counts(normalize=True)*100).reset_index()
status_summary.columns=['Status','Percentage']
status_summary

# 3. AOV Average Order Value
avg_order_value=orders_master['total_order_value'].mean()
print(f"The average customer spends: ${avg_order_value:.2f}")

#  New customers acquisition + Monthly Increase %
first_purchase=orders_master.groupby('customer_unique_id')['order_purchase_timestamp'].min().reset_index()
first_purchase.columns=['customer_unique_id','first_purchase_date']
first_purchase['month_year']=first_purchase['first_purchase_date'].dt.to_period('M')

new_customers_monthly=first_purchase.groupby('month_year').size().reset_index(name='new_customer_count')
new_customers_monthly['growth_rate%']=new_customers_monthly['new_customer_count'].pct_change()*100

display(new_customers_monthly.style.format({'growth_rate%':'{:.2f}%'}))

  customer_state  total_order_value
0             SP         5921678.12
1             RJ         2129681.98
2             MG         1856161.49
3             RS          885826.76
4             PR          800935.44
5             BA          611506.67
6             SC          610213.60
7             DF          353229.44
8             GO          347706.93
9             ES          324801.91
The average customer spends: $159.33


,month_year,new_customer_count,growth_rate%
0,2016-09,4,nan%
1,2016-10,321,7925.00%
2,2016-12,1,-99.69%
3,2017-01,764,76300.00%
4,2017-02,1752,129.32%
5,2017-03,2636,50.46%
6,2017-04,2352,-10.77%
7,2017-05,3596,52.89%
8,2017-06,3139,-12.71%
9,2017-07,3894,24.05%


In [28]:
orders_master

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,approval_delay,total_order_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,promised_delivery_time,delivery_time
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0.178333,38.71,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,15,8.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,30.713889,141.46,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,19,13.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,0.276111,179.12,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,26,9.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,0.298056,72.20,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,26,13.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.030556,28.62,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,12,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,0.000000,85.08,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,18,8.0
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,0.194167,195.00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,23,22.0
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,0.292500,271.01,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,30,24.0
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,0.131667,441.16,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,37,17.0
